## JSON Exercise Solution

This notebook provides the answers to the following three inquiries regarding the `'world_bank_projects.json'` database, which is a database of World Bank school improvement projects:

1. Find the 10 countries with the most projects.  
2. Find the top 10 major project themes (using the `'mjtheme_namecode'` from the `json` database)  
3. Create a database with the missing `'name'` values from the `'mjtheme_namecode'` column filled in with the correct information.

As an intial step, we will `import` the necessary python packages as follows:

In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

### Inquiry 1 - Find the 10 countries with the most projects

We can answer this inquiry by running a straightforward `value_counts()` on the `'countryname'` column of the database.

In [2]:
df = pd.read_json('data/world_bank_projects.json')

In [4]:
df['countryname'].value_counts().head(15)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Republic of Mozambique             11
Africa                             11
Burkina Faso                        9
Federative Republic of Brazil       9
Islamic Republic of Pakistan        9
United Republic of Tanzania         8
Republic of Tajikistan              8
Name: countryname, dtype: int64

Curiously, tied for the 9th most common result in the `'countryname'` column is `'Africa'`, which of course is a continent made up of numerous countries.  We can do some additional investigation to determine which specific countries within Africa were involved in the 11 projects.  We can start by populating a database with only the 11 Africa projects and see if there are other columns that may provide additional country information:

In [6]:
df.africa = df[df.countryname == 'Africa']
print(df.africa.columns)

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')


Let's see if the `borrower'` or `'countrycode'` columns provide more specificity:

In [7]:
print(df.africa[['borrower', 'countrycode']])

                          borrower countrycode
45                          ECOWAS          3A
46                  UGANDA-COMOROS          3A
51                OSS, IUCN, CILSS          3A
58                   BANK EXECUTED          3A
65         BURUNDI,RWANDA,TANZANIA          3A
99              GOVERNMENT OF MALI          3A
167  MINISTRIES OF TELECOM AND ICT          3A
184                       TANZANIA          3A
287     MALAWI, MOZAMBIQUE, ZAMBIA          3A
353          NILE BASIN INITIATIVE          3A
449            GOVERNMENT OF NIGER          3A


While the '`countrycode'` column is unhelpful, we can see from the `'borrower'` column that a number of different African countries make up the 11 projects.  We therefore should not include that entry in the inquiry answer.

### Inquiry 1 Answer - the 10 countries with the most projects:

|             Country             | Projects |
|:-------------------------------:|:--------:|
|    People's Republic of China   |    19    |
|      Republic of Indonesia      |    19    |
|  Socialist Republic of Vietnam  |    17    |
|        Republic of India        |    16    |
|        Republic of Yemen        |    13    |
|              Nepal              |    12    |
| People's Republic of Bangladesh |    12    |
|        Kingdom of Morocco       |    12    |
|      Republic of Mozambique     |    11    |
|           Burkina Faso          |     9    |
|  Federative Republic of Brazil  |     9    |
|   Islamic Republic of Pakistan  |     9    |

### Inquiry 2 - Find the top 10 major project themes (using the `'mjtheme_namecode'` from the `json` database)

As an initial step, we can display the `'mjtheme_namecode'` column from the existing `df` to get an understanding of the type of data contained in that column:

In [8]:
df['mjtheme_namecode'].head(10)

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
5    [{'code': '6', 'name': 'Social protection and ...
6    [{'code': '2', 'name': 'Public sector governan...
7    [{'code': '11', 'name': 'Environment and natur...
8    [{'code': '10', 'name': 'Rural development'}, ...
9    [{'code': '2', 'name': 'Public sector governan...
Name: mjtheme_namecode, dtype: object

It looks like the values imported from the `'mjtheme_namecode'` column in the `'json'` file are lists of dictionaries.  Let's normalize the values in a standalone database using `json_normalize`:

In [10]:
df_themes = json.load((open('data/world_bank_projects.json')))
df_themes_norm = json_normalize(df_themes, 'mjtheme_namecode')
df_themes_norm.head(10)

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion


Much better!  Now that we have a discrete dataset, we can determine the most popular project themes simply by running a `value_counts()` on the `'name'` column:

In [12]:
df_themes_norm['name'].value_counts()

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Rule of law                                      12
Name: name, dtype: int64

It looks like the counts are incomplete because there are 122 blank entries in the `'name'` column.  As a workaround, we could cross-reference the names from the corresponding `'code'` column, which is not missing any values:

In [15]:
df_themes_norm['code'].value_counts().head(10)

11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: code, dtype: int64

Instead, however, we can address inquiry 3 by filling in the missing values in the `'name'` column and then re-running a `'value_counts()'` on the complete database. 

### Inquiry 3 - Create a database with the missing 'name' values from the 'mjtheme_namecode' column filled in with the correct information

As we saw above, the `'name'` column is missing 122 entries.  We should be able to isolate those entries using the `sort_values()` method.  We do not want all of the blank entries isolated together, however, as we will need to populate them with different values depending on the corresponding value in the `'code'` column.  We accordingly need to sort by the `'code'` values first, and then by the `'name'` values:  

In [17]:
df_themes_norm = df_themes_norm.sort_values(['code', 'name'])
df_themes_norm.head(50)

,code,name
212,1,
363,1,
1024,1,
1114,1,
1437,1,
2,1,Economic management
88,1,Economic management
175,1,Economic management
204,1,Economic management
205,1,Economic management


Now that we have the data sorted, we can backfill the blank entries to populate the correct project theme name:

In [21]:
df_themes_norm = df_themes_norm.fillna(method='bfill')
df_themes_norm.head(50)

,code,name
212,1,
363,1,
1024,1,
1114,1,
1437,1,
2,1,Economic management
88,1,Economic management
175,1,Economic management
204,1,Economic management
205,1,Economic management


Well that didn't work.  It looks like the `fillna()` method only recognizes `NaN` values.  Let's convert the blank entries to `NaN` values using `np.nan` and try again:

In [23]:
df_themes_norm['name'][df_themes_norm['name'] == ""] = np.nan
df_themes_norm.head(50)

,code,name
212,1,NaN
363,1,NaN
1024,1,NaN
1114,1,NaN
1437,1,NaN
2,1,Economic management
88,1,Economic management
175,1,Economic management
204,1,Economic management
205,1,Economic management


In [24]:
df_themes_norm = df_themes_norm.fillna(method='bfill')
df_themes_norm.head(50)

,code,name
212,1,Economic management
363,1,Economic management
1024,1,Economic management
1114,1,Economic management
1437,1,Economic management
2,1,Economic management
88,1,Economic management
175,1,Economic management
204,1,Economic management
205,1,Economic management


It worked!  Now a simple `value_counts()` on the `'name'` column should provide us with the top 10 project themes:

In [30]:
df_themes_norm['name'].value_counts().head(10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64

### Inquiry 2 Answer - the 10 most popular project themes:

|                Project Theme               | Projects |
|:------------------------------------------:|:--------:|
| Environmental/Natural Resources Management |    250   |
|              Rural Development             |    216   |
|              Human Development             |    210   |
|          Public Sector Governance          |    199   |
|    Social Protection and Risk Management   |    168   |
|   Financial and Private Sector Management  |    146   |
|         Social dev/gender/inclusion        |    130   |
|            Trade and Integration           |    77    |
|              Urban Development             |    50    |
|             Economic Management            |    38    |